***
____
![revit](https://i.ibb.co/bQ3dB8C/curso-revit.png)

***
***


# Clase 07
## Selección de variables

In [3]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]

Para garantizar la reproducibilidad (esto es, que al ejecutar este notebook otra vez los resultados sean idénticos) vamos a fijar la semilla que usa scikit learn para hacer particiones aleatorias. Para esto tenemos que fijar la semilla (seed) cada vez que llamemos a la aplicacion.

In [4]:
 np.random.seed(42)

# Selección de variables

En esta sección vamos a ver las distintas técnicas de selección de variables, que son las técnicas que nos permiten filtrar las variables que tenemos y elegir aquellas que realmente están ayudando al modelo.

¿Por qué querriamos eliminar variables? Generalmente, cuanta más información tenemos sobre cada observación mejor será la calidad de nuestras predicciones (cuanto más sepamos, más facil será diferenciar unos casos de otros). No obstante, esto no tiene por que ser así en todas las situaciones, hay veces que ciertas variables pueden ser contraproducentes y hacer que nuestros modelos funcionen peor, por ejemplo debido a errores de medición.

Seleccionar variables tiene varias ventajas. En primer lugar, como hemos dicho, puede mejorar las puntuaciones de los modelos (eliminando parte del ruido en los datos), por otra parte, reducir el número de variables independientes reduce la complejidad del modelo, reduciendo así el efecto de sobreajuste del modelo. Además, tener menos variables independientes y modelos más simples significa que nuestros modelos son más rápidos de entrenar, y podemos entrenarlos con más datos en las mismas máquinas.

## Cargamos los datos

Cargaremos los datos del data `Caso mora en tarjetas de crédito`

In [8]:
# datos = pd.read_csv("../caso_TC.csv").drop(["ID"], axis = 1)
datos = pd.read_csv("../ames.csv").drop(["id_parcela"], axis = 1)

In [9]:
datos.shape

(2930, 80)

In [10]:
datos.head()

,tipo_construccion,tipo_zona,perimetro_conectado_calle,area_parcela,tipo_acceso,tipo_acceso_trasero,forma_parcela,LandContour,tipo_instalaciones,config_parcela,...,area_piscina,calidad_piscina,calidad_valla,atributo_miscelaneo,valor_atributo_miscelaneo,mes_venta,ano_venta,tipo_venta,condicion_venta,precio_venta
0,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [11]:
datos.sample(1).T

,1357
tipo_construccion,70
tipo_zona,RM
perimetro_conectado_calle,NaN
area_parcela,5100
tipo_acceso,Pave
tipo_acceso_trasero,Grvl
forma_parcela,Reg
LandContour,Lvl
tipo_instalaciones,AllPub
config_parcela,Inside


In [7]:
# datos.VAR13 = datos.VAR13.astype('category',copy=False)

# import datetime as dt

# datos.VAR04 = pd.to_datetime(datos.VAR04)
# datos["anio_ingreso"] = datos.VAR04.dt.year
# datos["tiemp_trab"] = 2019 - datos.anio_ingreso

# datos = datos.drop(["VAR04", "anio_ingreso"], axis = 1)

In [12]:
datos.dtypes

tipo_construccion                int64
tipo_zona                       object
perimetro_conectado_calle      float64
area_parcela                     int64
tipo_acceso                     object
tipo_acceso_trasero             object
forma_parcela                   object
LandContour                     object
tipo_instalaciones              object
config_parcela                  object
pendiente_parcela               object
barrio                          object
cercania_carretera1             object
cercania_carretera2             object
tipo_edificio                   object
tipo_casa                       object
calidad_general                  int64
condicion_general                int64
fecha_construccion               int64
ano_remodelado                   int64
tipo_tejado                     object
material_tejado                 object
material_exterior1              object
material_exterior2              object
tipo_revestimiento              object
area_revestimiento       

In [13]:
objetivo = "precio_venta"

## Procesado de datos

In [14]:
# Separamos los datos numéricos y categóricos
datos_numericos = datos.select_dtypes(include=['float64', "int64"])
datos_categoricos = datos.select_dtypes(exclude=['float64', "int64"])

# Para los missing numéricos los imputamos con la media
for col in datos_numericos.columns:
    datos_numericos[col].fillna(datos_numericos[col].mean(), inplace=True)

# Para los cat    
for col in datos_categoricos.columns:
    datos_categoricos[col].fillna("Otros", inplace=True)

# Para los categoricos creamos dummies
datos_categoricos_codificados = pd.get_dummies(datos_categoricos)
df_final = pd.concat([datos_numericos, datos_categoricos_codificados], axis=1)

C:\Users\Alfy\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
df_final.shape

(2930, 321)

Ya tenemos un dataset preparado para poder entrenar modelos. Este dataset tiene una complejidad dimensional alta ( 986 variables independientes), por lo que vamos a usar técnicas de selección de variables para reducirla.

Antes que nada vamos a ver que errores obtenemos con diversos modelos entrenando con el dataset con todas las variables.

In [17]:
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

X=df_final.drop(objetivo, axis=1)
y=df_final[objetivo]

Vamos a usar la función `cross_validate` que es una versión más flexible que `cross_val_score`. Evaluaremos usando el área bajo la curva ROC

In [21]:
clasificador = cross_validate(LinearRegression(), X, y, scoring="neg_mean_squared_error", n_jobs=-1, cv=10)

Dicha función devuelve no solo las puntuaciones en los datos de testing, sino también en los datos de entrenamiento. Además, nos devuelve el tiempo que tarda dicho estimador en ajustar el modelo y en hacer predicciones con el mismo.

In [22]:
pd.DataFrame(clasificador)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,fit_time,score_time,test_score,train_score
0,1.109660,0.006981,-5.199446e+08,-4.104954e+08
1,1.092701,0.007983,-2.519164e+13,-3.891820e+08
2,1.091706,0.006985,-3.650098e+08,-4.298505e+08
3,1.076744,0.006985,-6.767508e+08,-4.045950e+08
4,0.205449,0.002997,-4.912438e+08,-4.265764e+08
5,0.151596,0.002996,-3.234677e+16,-4.034457e+08
6,0.153589,0.003990,-8.139520e+08,-4.007726e+08
7,0.157425,0.002994,-2.320506e+09,-3.555015e+08
8,0.118682,0.001997,-6.781527e+08,-4.075585e+08
9,0.114695,0.001995,-7.625317e+15,-4.297518e+08


In [57]:
def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="neg_mean_squared_error", n_jobs=-1, cv=10, return_train_score=True)
    return resultados_estimador

In [71]:
resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    resultados_df.test_score = (-resultados_df.test_score)**0.5
    resultados_df.train_score = (-resultados_df.train_score)**0.5
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [59]:
resultados["reg_logis_sin_seleccion"] = evaluar_modelo(LinearRegression(), X, y)
resultados["knn_sin_seleccion"] = evaluar_modelo(KNeighborsRegressor(n_neighbors=500, weights="distance"), X, y)
resultados["rf_sin_seleccion"] = evaluar_modelo(RandomForestRegressor(n_estimators=100), X, y)

In [72]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.155542,0.002584,2.723871e+07,20136.803471,1.210461,1.000000,1007.771361,330.586783
knn_sin_seleccion,0.128498,0.475827,6.108471e+04,60.912307,1.000000,184.127853,2.259998,1.000000
rf_sin_seleccion,16.967831,0.024469,2.702866e+04,9659.167554,132.046933,9.468687,1.000000,158.574976


Hay 3 tipos generales de estrategias de selección de variables:

# Métodos de filtrado

Los métodos de filtrado usan métodos estadísticos para seleccionar las variables que proporcionan la mayor cantidad de información. Estos métodos se aplican de forma previa a entrenar el modelo (preprocesado), y **son completamente independientes de la elección del estimador**. Generalmente funcionan definiendo una función de evaluación $S(xk_i, y_k)$, evaluando cada variable independiente para cada observación respecto a la variable objetivo de dicha observación, y eligiendo aquellas `K` variables que mejor funcionan.



Scikit-learn tiene las siguientes funciones de evaluación:

- Para regresión: `f_regression, mutual_info_regression`
- Para clasificación: `chi2, f_classif, mutual_info_classif`

f_regression y f_classif devuelven [estadísticos F](https://es.wikipedia.org/wiki/Estad%C3%ADstico_F) (F-values), entrenando un modelo lineal entre las variables independientes y la objetivo en el caso de regresión, y un test ANOVA en el caso de clasificación.

`mutual_info_regression` y `mutual_info_classif` computan el coeficiente de información mutua (MIC) entre las variables independientes y la variable objetivo.

El coeficiente de información mutua nos da una medida de la dependencia entre las variables. El MIC entre dos variables es 0 si no hay relación entre las mismas, y aumenta conforme más relación tienen.

El MIC se define como:

$$I(X;Y) = \sum_{x,y} P_{XY}(x,y) \log
{P_{XY}(x,y) \over P_X(x) P_Y(y)}$$

con $P_{XY}$ siendo la probabilidad conjunta de X e Y


El evaluador `chi2` calcula el estadístico chi cuadrado y lo convierte a estadísticos F, con el estadístico chi cuadrado definido como:
  $$\chi^2(x,y) = \sum {\frac{(x-y)^2}{y}}$$


In [30]:
from sklearn.feature_selection import SelectKBest, f_regression

In [31]:
selector_kbest10 = SelectKBest(f_regression, k=10)
X_kbest10 = selector_kbest10.fit_transform(X, y)

In [32]:
X_kbest10.shape

(2930, 10)

La funcion `get_support` nos devuelve un vector booleano (True/False), aquellos elementos con True son las columnas que se han seleccionado

In [33]:
columnas_seleccion_kbest10 = X.loc[:,selector_kbest10.get_support()].columns
columnas_seleccion_kbest10

Index(['calidad_general', 'fecha_construccion', 'area_sotano_total',
       'area_piso1', 'area_habitable_sobre_suelo', 'n_aseos_sobre_suelo',
       'n_coches_garaje', 'area_garage', 'calidad_material_exterior_TA',
       'altura_sotano_Ex'],
      dtype='object')

El parámetro `scores_` del selector nos devuelve los resultados de la función de evaluación

In [34]:
selector_kbest10.scores_[:10]

array([2.13550300e+01, 3.84634199e+02, 2.23940611e+02, 5.17874808e+03,
       3.05986139e+01, 1.32682270e+03, 1.16173403e+03, 1.00651895e+03,
       6.74854032e+02, 1.01573853e-01])

In [45]:
evaluacion_kbest10 = pd.DataFrame({"variable":X.columns, 
                                   "Score":selector_kbest10.scores_, 
                                   "Seleccionado":selector_kbest10.get_support()})

In [48]:
evaluacion_kbest10[evaluacion_kbest10.Seleccionado==True]

,variable,Score,Seleccionado
3,calidad_general,5178.748077,True
5,fecha_construccion,1326.822701,True
11,area_sotano_total,1948.398606,True
12,area_piso1,1844.469327,True
15,area_habitable_sobre_suelo,2922.592376,True
18,n_aseos_sobre_suelo,1241.061221,True
25,n_coches_garaje,2117.874880,True
26,area_garage,2035.498773,True
181,calidad_material_exterior_TA,1570.799109,True
193,altura_sotano_Ex,1592.763109,True


Esto nos permite ver cual es la puntuación que le da el evaluador `f_regression` a cada variable independiente

In [73]:
resultados["reg_logis_kbest_10"] = evaluar_modelo(LinearRegression(), X_kbest10, y)
resultados["knn_kbest_10"] = evaluar_modelo(KNeighborsRegressor(n_neighbors=500, weights="distance"), X_kbest10, y)
resultados["rf_kbest_10"] = evaluar_modelo(RandomForestRegressor(n_estimators=100), X_kbest10, y)

In [74]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.155542,0.002584,2.723871e+07,20136.803471,37.025249,2.962123,1007.771361,330.586783
knn_sin_seleccion,0.128498,0.475827,6.108471e+04,60.912307,30.587718,545.409324,2.259998,1.000000
rf_sin_seleccion,16.967831,0.024469,2.702866e+04,9659.167554,4039.014290,28.047415,1.000000,158.574976
reg_logis_kbest_10,0.004859,0.000872,3.393810e+04,33974.229189,1.156707,1.000000,1.255634,557.756405
knn_kbest_10,0.004201,0.092579,4.709470e+04,1640.506201,1.000000,106.116774,1.742399,26.932262
rf_kbest_10,2.308373,0.021196,3.003638e+04,10779.916940,549.484069,24.295283,1.111279,176.974368


Vemos que al reducir las dimensiones a solo 10, los modelos lineales y RF funcionan bastante peor, aunque vemos que por otra parte las diferencias entre la evaluación de entrenamiento y test se han reducido en todos los casos. Esto es así por que al tener menos variables, hay mucha menos flexibilidad para sobreajustar (es decir, "memorizar" los datos de entrenamiento). SVM sin embargo funciona más o menos igual (de mal), pero los tiempos de entrenamiento se reducen considerablemente (de 3.9 segundos a 0.7).

Probamos ahora para 50 variables

In [75]:
selector_kbest50 = SelectKBest(f_regression, k=50)
X_kbest50 = selector_kbest50.fit_transform(X, y)

In [76]:
resultados["reg_logis_kbest_50"] = evaluar_modelo(LinearRegression(), X_kbest50, y)
resultados["knn_kbest_50"] = evaluar_modelo(KNeighborsRegressor(n_neighbors=500, weights="distance"), X_kbest50, y)
resultados["rf_kbest_50"] = evaluar_modelo(RandomForestRegressor(n_estimators=100), X_kbest50, y)

In [77]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.155542,0.002584,2.723871e+07,20136.803471,37.025249,2.962123,1007.771361,330.586783
knn_sin_seleccion,0.128498,0.475827,6.108471e+04,60.912307,30.587718,545.409324,2.259998,1.000000
rf_sin_seleccion,16.967831,0.024469,2.702866e+04,9659.167554,4039.014290,28.047415,1.000000,158.574976
reg_logis_kbest_10,0.004859,0.000872,3.393810e+04,33974.229189,1.156707,1.000000,1.255634,557.756405
knn_kbest_10,0.004201,0.092579,4.709470e+04,1640.506201,1.000000,106.116774,1.742399,26.932262
rf_kbest_10,2.308373,0.021196,3.003638e+04,10779.916940,549.484069,24.295283,1.111279,176.974368
reg_logis_kbest_50,0.060456,0.002675,3.012057e+04,29341.100150,14.390847,3.066053,1.114394,481.694123
knn_kbest_50,0.022614,0.202372,4.861514e+04,145.431771,5.382924,231.965430,1.798651,2.387560
rf_kbest_50,5.821248,0.022152,2.771552e+04,10103.945416,1385.687160,25.390932,1.025412,165.876913


Vemos que para 50 variables, los tres estimadores funcionan de forma similar al modelo entrenado con las 209 variables iniciales, sin embargo los tiempos de entrenamiento de los mismos se reducen considerablemente.

De forma similar a `SelectKBest`, donde le decimos al transformador el número de variables que queremos mantener, podemos usar [`SelectPercentile`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile) donde especificamos el porcentaje de variables sobre el total.

Por ejemplo, si solo quisieramos conservar el 10% de las variables (sin importar que tengamos 200 o 20000), podemos hacer lo siguiente:


In [78]:
from sklearn.feature_selection import SelectPercentile

selector_pct10 = SelectPercentile(f_regression, percentile=10)
X_pct10 = selector_pct10.fit_transform(X, y)

In [79]:
X_pct10.shape

(2930, 32)

El transformador nos selecciona 98 variables (el 10 por ciento de 981 redondeado)

# Métodos envolventes (wrapper methods)

Los métodos envolventes *(wrapper methods)* funcionan de forma similar a los métodos de ranking. Sin embargo, en lugar de usar una función estadística independiente del modelo para evaluar las variables, estos métodos usan la función de evaluación o el performance de los modelos como input para decidir que variables elegir (es decir, "envuelven" el funcionamiento del estimador). Ésto significa que los métodos de filtrado se pueden aplicar independientemente de la elección del modelo, ya que consideran los modelos como una caja negra que produce evaluaciones, aunque claro, diferentes modelos producirán diferentes selecciones de variables.

`Scikit-learn` implementa un metodo envolvente llamado **Recursive Feature Elimination [(RFE)](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)**, o Eliminación Recursiva de Variables. RFE funciona seleccionando todas las variables, entrenando el modelo, usando los coeficientes `coef_` o la importancia de las variables `feature_importances_` en función del estimador, y eliminando n variables. Este proceso se repite hasta que se alcanza el número de variables deseado.

In [81]:
from sklearn.feature_selection import RFE
estimador_selector = RandomForestRegressor()
selector_rfe10_rf = RFE(estimador_selector, n_features_to_select=10)
X_rfe10_rf = selector_rfe10_rf.fit_transform(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

In [82]:
X_rfe10_rf.shape

(2930, 10)

In [83]:
evaluacion_rfe10_rf = sorted(
    filter(lambda c: c[2], 
        zip(
            X.columns,
            selector_rfe10_rf.ranking_,
            selector_rfe10_rf.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)

In [84]:
evaluacion_rfe10_rf

[('area_parcela', 1, True),
 ('calidad_general', 1, True),
 ('fecha_construccion', 1, True),
 ('ano_remodelado', 1, True),
 ('area_sotano_habitable1', 1, True),
 ('area_sotano_total', 1, True),
 ('area_piso1', 1, True),
 ('area_piso2', 1, True),
 ('area_habitable_sobre_suelo', 1, True),
 ('area_garage', 1, True)]

In [85]:
resultados["reg_lineal_rfe10_rf"] = evaluar_modelo(LinearRegression(), X_rfe10_rf, y)
resultados["knn_rfe10"] = evaluar_modelo(KNeighborsRegressor(n_neighbors=500, weights="distance"), X_rfe10_rf, y)
resultados["rf_rfe10_rf"] = evaluar_modelo(RandomForestRegressor(n_estimators=100), X_rfe10_rf, y)


In [86]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.155542,0.002584,2.723871e+07,20136.803471,37.025249,2.962123,1007.771361,330.586783
knn_sin_seleccion,0.128498,0.475827,6.108471e+04,60.912307,30.587718,545.409324,2.259998,1.000000
rf_sin_seleccion,16.967831,0.024469,2.702866e+04,9659.167554,4039.014290,28.047415,1.000000,158.574976
reg_logis_kbest_10,0.004859,0.000872,3.393810e+04,33974.229189,1.156707,1.000000,1.255634,557.756405
knn_kbest_10,0.004201,0.092579,4.709470e+04,1640.506201,1.000000,106.116774,1.742399,26.932262
rf_kbest_10,2.308373,0.021196,3.003638e+04,10779.916940,549.484069,24.295283,1.111279,176.974368
reg_logis_kbest_50,0.060456,0.002675,3.012057e+04,29341.100150,14.390847,3.066053,1.114394,481.694123
knn_kbest_50,0.022614,0.202372,4.861514e+04,145.431771,5.382924,231.965430,1.798651,2.387560
rf_kbest_50,5.821248,0.022152,2.771552e+04,10103.945416,1385.687160,25.390932,1.025412,165.876913
reg_lineal_rfe10_rf,0.007858,0.001470,3.482472e+04,34837.322294,1.870586,1.684849,1.288437,571.925843


Si usamos otro estimador para evaluar veremos que las variables elegidas pueden ser completamente distintas. Los estimadores que se pueden usar tienen que implementar el metodo `coef_` o el metodo `feature_importance` (es decir, tienen que tener una manera de ordenar variables en función de su importancia). Por ejemplo, no podemos usar SVMs.

In [87]:
estimador_selector = LinearRegression()
selector_rfe10_lineal = RFE(estimador_selector, n_features_to_select=10)
X_rfe10_lineal = selector_rfe10_lineal.fit_transform(X, y)

In [88]:
evaluacion_rfe10_lineal = sorted(
    filter(lambda c: c[2], 
        zip(
            X.columns,
            selector_rfe10_lineal.ranking_,
            selector_rfe10_lineal.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)
evaluacion_rfe10_lineal

[('tipo_revestimiento_BrkCmn', 1, True),
 ('tipo_revestimiento_BrkFace', 1, True),
 ('tipo_revestimiento_CBlock', 1, True),
 ('tipo_revestimiento_None', 1, True),
 ('tipo_revestimiento_Otros', 1, True),
 ('tipo_revestimiento_Stone', 1, True),
 ('tipo_andamios_CBlock', 1, True),
 ('tipo_andamios_PConc', 1, True),
 ('tipo_andamios_Stone', 1, True),
 ('tipo_andamios_Wood', 1, True)]

In [89]:
set(evaluacion_rfe10_rf).intersection(set(evaluacion_rfe10_lineal))

set()

In [90]:
resultados["reg_lineal_rfe10_lineal"] = evaluar_modelo(LinearRegression(), X_rfe10_lineal, y)
resultados["knn_rfe10_lineal"] = evaluar_modelo(KNeighborsRegressor(n_neighbors=500, weights="distance"), X_rfe10_lineal, y)
resultados["rf_rfe10_lineal"] = evaluar_modelo(RandomForestRegressor(n_estimators=100), X_rfe10_lineal, y)


In [91]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.155542,0.002584,2.723871e+07,20136.803471,60.377624,3.182979,1.007771e+03,330.586783
knn_sin_seleccion,0.128498,0.475827,6.108471e+04,60.912307,49.879845,586.075177,2.259998e+00,1.000000
rf_sin_seleccion,16.967831,0.024469,2.702866e+04,9659.167554,6586.480546,30.138637,1.000000e+00,158.574976
reg_logis_kbest_10,0.004859,0.000872,3.393810e+04,33974.229189,1.886258,1.074560,1.255634e+00,557.756405
knn_kbest_10,0.004201,0.092579,4.709470e+04,1640.506201,1.630715,114.028867,1.742399e+00,26.932262
rf_kbest_10,2.308373,0.021196,3.003638e+04,10779.916940,896.051827,26.106745,1.111279e+00,176.974368
reg_logis_kbest_50,0.060456,0.002675,3.012057e+04,29341.100150,23.467368,3.294658,1.114394e+00,481.694123
knn_kbest_50,0.022614,0.202372,4.861514e+04,145.431771,8.778014,249.260829,1.798651e+00,2.387560
rf_kbest_50,5.821248,0.022152,2.771552e+04,10103.945416,2259.660617,27.284087,1.025412e+00,165.876913
reg_lineal_rfe10_rf,0.007858,0.001470,3.482472e+04,34837.322294,3.050392,1.810472,1.288437e+00,571.925843


Vemos que el estimador RF entrenado con solo 10 variables seleccionadas por `RFE` basado en RF es el modelo que mejor funciona, y tarda 8 veces menos en entrenar.

scikit-learn también tiene la implementación [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) que implementa el mismo algoritmo pero hace validación cruzada y entrena en múltiples particiones del dataset para calcular automáticamente el conjunto de variables que funciona mejor (a costa de un mayor tiempo de procesamiento).

In [92]:
from sklearn.feature_selection import RFECV

estimador_selector = RandomForestRegressor()
selector_rfecv = RFECV(estimador_selector)
X_rfecv = selector_rfecv.fit_transform(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-p

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

...

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

In [93]:
X_rfecv.shape

(2930, 286)

En este caso `RFECV` ha elegido 188 variables como aquellas que proporcionan el mejor funcionamiento al estimador RF utilizado.

In [94]:
resultados["reg_lineal_rfecv_rf"] = evaluar_modelo(LinearRegression(), X_rfecv, y)
resultados["knn_rfecv_rf"] = evaluar_modelo(KNeighborsRegressor(n_neighbors=500, weights="distance"), X_rfecv, y)
resultados["rf_rfecv_rf"] = evaluar_modelo(RandomForestRegressor(n_estimators=100), X_rfecv, y)

In [95]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.155542,0.002584,2.723871e+07,20136.803471,60.377624,3.182979,1.007771e+03,330.586783
knn_sin_seleccion,0.128498,0.475827,6.108471e+04,60.912307,49.879845,586.075177,2.259998e+00,1.000000
rf_sin_seleccion,16.967831,0.024469,2.702866e+04,9659.167554,6586.480546,30.138637,1.000000e+00,158.574976
reg_logis_kbest_10,0.004859,0.000872,3.393810e+04,33974.229189,1.886258,1.074560,1.255634e+00,557.756405
knn_kbest_10,0.004201,0.092579,4.709470e+04,1640.506201,1.630715,114.028867,1.742399e+00,26.932262
rf_kbest_10,2.308373,0.021196,3.003638e+04,10779.916940,896.051827,26.106745,1.111279e+00,176.974368
reg_logis_kbest_50,0.060456,0.002675,3.012057e+04,29341.100150,23.467368,3.294658,1.114394e+00,481.694123
knn_kbest_50,0.022614,0.202372,4.861514e+04,145.431771,8.778014,249.260829,1.798651e+00,2.387560
rf_kbest_50,5.821248,0.022152,2.771552e+04,10103.945416,2259.660617,27.284087,1.025412e+00,165.876913
reg_lineal_rfe10_rf,0.007858,0.001470,3.482472e+04,34837.322294,3.050392,1.810472,1.288437e+00,571.925843


Vemos que el método que funciona mejor es el de RFECV usando RF como estimador. Este método funciona mejor que  el segundo mejor modelo, un Random Forest entrenado con todo el dataset, pero se entrena en menos tiempo (0.81 segundos versus  0.99 segundos)

# Metodos embebidos (Embedded Methods)

Éstos métodos son metodos de selección de variables que están integrados dentro del proceso de entrenamiento de modelos, y son específicos para cada modelo.

El ejemplo clásico de método embebidos de selección de variables son los procesos de regularización en regresión lineal. Por ejemplo el método L1 tiende a convertir los coeficientes de las variables que no funcionan a 0, por lo tanto eliminandolas.